In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# TFX Pipeline from Pre-build Components


This pipeline demonstrates the TFX capabilities at scale. The pipeline uses a public BigQuery dataset and uses GCP services to preprocess data (Dataflow) and train the model (Cloud ML Engine). The model is then deployed to kubeflow for prediction service.

In [1]:
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp.components as comp
import datetime

import kubernetes as k8s

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/kubeflow-pipeline-fantasy.json'

In [4]:
# Required Parameters
PROJECT_ID='kubeflow-pipeline-fantasy'
GCS_BUCKET='gs://kubeflow-pipeline-ui'

## Create client

If you run this notebook **outside** of a Kubeflow cluster, run the following command:
- `host`: The URL of your Kubeflow Pipelines instance, for example "https://`<your-deployment>`.endpoints.`<your-project>`.cloud.goog/pipeline"
- `client_id`: The client ID used by Identity-Aware Proxy
- `other_client_id`: The client ID used to obtain the auth codes and refresh tokens.
- `other_client_secret`: The client secret used to obtain the auth codes and refresh tokens.

```python
client = kfp.Client(host, client_id, other_client_id, other_client_secret)
```

If you run this notebook **within** a Kubeflow cluster, run the following command:
```python
client = kfp.Client()
```

You'll need to create OAuth client ID credentials of type `Other` to get `other_client_id` and `other_client_secret`. Learn more about [creating OAuth credentials](
https://cloud.google.com/iap/docs/authentication-howto#authenticating_from_a_desktop_app)

In [5]:
# Optional Parameters, but required for running outside Kubeflow cluster

# The host for full deployment of Kubeflow ends with '/pipeline'
HOST = 'https://kubeflow-st-ui.endpoints.kubeflow-pipeline-fantasy.cloud.goog/pipeline'
# Full deployment of Kubeflow on GCP is usually protected through IAP, therefore the following 
# will be needed to access the endpoint
CLIENT_ID = "493831447550-os23o55235htd9v45a9lsejv8d1plhd0.apps.googleusercontent.com"
OTHER_CLIENT_ID = "493831447550-iu24vv6id3ng5smhf2lboovv5qukuhbh.apps.googleusercontent.com"
OTHER_CLIENT_SECRET = "cB8Xj-rb9JWCYcCRDlpTMfhc"

# # The host for managed 'AI Platform Pipeline' ends with 'pipelines.googleusercontent.com'
# HOST = 'https://7c021d0340d296aa-dot-us-central2.pipelines.googleusercontent.com'

In [6]:
# This is to ensure the proper access token is present to reach the end point for managed 'AI Platform Pipeline'
# If you are not working with managed 'AI Platform Pipeline', this step is not necessary
! gcloud auth print-access-token

ya29.a0Adw1xeVU2xFIGr6OPo10YNC7n04mElu4u1hadTABZuRBnVL-AXJ26XqLW8TpP09xYQebVmlae92PC6-DKBi6DFWV9eIjd8hdnY3qvIQfH2WAbTuk_d3SFc9Eyaw8SH425GMYAh0Ty9djbg3Bz9m4BHfMg6YeTT9NF_9sFyEJGPkz


In [7]:
# Create kfp client
in_cluster = True
try:
  k8s.config.load_incluster_config()
except:
  in_cluster = False
  pass

if in_cluster:
    client = kfp.Client()
else:
    client = kfp.Client(host=HOST, 
                        client_id=CLIENT_ID,
                        other_client_id=OTHER_CLIENT_ID, 
                        other_client_secret=OTHER_CLIENT_SECRET)

## Load pre-build components

In [8]:
dataflow_tf_data_validation_op  = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/dataflow/tfdv/component.yaml')
dataflow_tf_transform_op        = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/dataflow/tft/component.yaml')
tf_train_op                     = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/kubeflow/dnntrainer/component.yaml')
dataflow_tf_model_analyze_op    = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/dataflow/tfma/component.yaml')
dataflow_tf_predict_op          = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/dataflow/predict/component.yaml')

confusion_matrix_op             = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/local/confusion_matrix/component.yaml')
roc_op                          = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/local/roc/component.yaml')

kubeflow_deploy_op              = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0b07e456b1f319d8b7a7301274f55c00fda9f537/components/kubeflow/deployer/component.yaml')

## Build and compile pipeline

In [9]:
@dsl.pipeline(
  name='TFX Taxi Cab Classification Pipeline Example',
  description='Example pipeline that does classification with model analysis based on a public BigQuery dataset.'
)
def taxi_cab_classification(
    output,
    project,
    column_names='gs://ml-pipeline-playground/tfx/taxi-cab-classification/column-names.json',
    key_columns='trip_start_timestamp',
    train='gs://ml-pipeline-playground/tfx/taxi-cab-classification/train.csv',
    evaluation='gs://ml-pipeline-playground/tfx/taxi-cab-classification/eval.csv',
    mode='local',
    preprocess_module='gs://ml-pipeline-playground/tfx/taxi-cab-classification/preprocessing.py',
    learning_rate=0.1,
    hidden_layer_size='1500',
    steps=3000,
    analyze_slice_column='trip_start_hour'
):
    output_template = str(output) + '/{{workflow.uid}}/{{pod.name}}/data'
    target_lambda = """lambda x: (x['target'] > x['fare'] * 0.2)"""
    target_class_lambda = """lambda x: 1 if (x['target'] > x['fare'] * 0.2) else 0"""

    tf_server_name = 'taxi-cab-classification-model-{{workflow.uid}}'

    validation = dataflow_tf_data_validation_op(
        inference_data=train,
        validation_data=evaluation,
        column_names=column_names,
        key_columns=key_columns,
        gcp_project=project,
        run_mode=mode,
        validation_output=output_template,
    )

    preprocess = dataflow_tf_transform_op(
        training_data_file_pattern=train,
        evaluation_data_file_pattern=evaluation,
        schema=validation.outputs['schema'],
        gcp_project=project,
        run_mode=mode,
        preprocessing_module=preprocess_module,
        transformed_data_dir=output_template
    )

    training = tf_train_op(
        transformed_data_dir=preprocess.output,
        schema=validation.outputs['schema'],
        learning_rate=learning_rate,
        hidden_layer_size=hidden_layer_size,
        steps=steps,
        optimizer='Adagrad',
        target='tips',
        preprocessing_module=preprocess_module,
        training_output_dir=output_template
    )

    analysis = dataflow_tf_model_analyze_op(
        model=training.output,
        evaluation_data=evaluation,
        schema=validation.outputs['schema'],
        gcp_project=project,
        run_mode=mode,
        slice_columns=analyze_slice_column,
        analysis_results_dir=output_template
    )

    prediction = dataflow_tf_predict_op(
        data_file_pattern=evaluation,
        schema=validation.outputs['schema'],
        target_column='tips',
        model=training.output,
        batch_size=32,
        run_mode=mode,
        gcp_project=project,
        predictions_dir=output_template
    )

    cm = confusion_matrix_op(
        predictions=prediction.output,
        target_lambda=target_lambda,
        output_dir=output_template
    )


    deploy = kubeflow_deploy_op(
        model_dir=str(training.output) + '/export/export',
        server_name=tf_server_name,
        cluster_name='kubeflow', 
        namespace='kubeflow',
        pvc_name='', 
        service_type='ClusterIP'
    )


    steps = [validation, preprocess, training, analysis, prediction, cm, deploy]
    for step in steps:
        step.apply(gcp.use_gcp_secret('user-gcp-sa'))

## Submit job

In [10]:
pipeline_func = taxi_cab_classification

In [11]:
experiment_name = 'TFX-From-Components'

arguments = {"project":PROJECT_ID,
             "output": GCS_BUCKET}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

/Users/luoshixin/LocalSim/virtualPython35/lib/python3.5/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "Float" based on the value "0.1".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/Users/luoshixin/LocalSim/virtualPython35/lib/python3.5/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "Integer" based on the value "3000".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
INFO:root:Creating experiment TFX-From-Components.
